# Requirements

In [4]:
import os,sys

import numpy as np

import librosa
import librosa.display

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt

import pandas as pd
from tqdm import trange
import librosa
import tqdm
from sklearn.model_selection import KFold

from datetime import datetime

import pathlib

#### Used for listening to audio samples in ipynb

In [2]:
import IPython.display
import numpy as np
import json

def Audio(audio: np.ndarray, sr: int):
    """
    Use instead of IPython.display.Audio as a workaround for VS Code.
    `audio` is an array with shape (channels, samples) or just (samples,) for mono.
    """

    if np.ndim(audio) == 1:
        channels = [audio.tolist()]
    else:
        channels = audio.tolist()

    return IPython.display.HTML("""
        <script>
            if (!window.audioContext) {
                window.audioContext = new AudioContext();
                window.playAudio = function(audioChannels, sr) {
                    const buffer = audioContext.createBuffer(audioChannels.length, audioChannels[0].length, sr);
                    for (let [channel, data] of audioChannels.entries()) {
                        buffer.copyToChannel(Float32Array.from(data), channel);
                    }
            
                    const source = audioContext.createBufferSource();
                    source.buffer = buffer;
                    source.connect(audioContext.destination);
                    source.start();
                }
            }
        </script>
        <button onclick="playAudio(%s, %s)">Play</button>
    """ % (json.dumps(channels), sr))

# Setting Path variables

In [11]:
base_path = pathlib.Path('D:/전재원졸업연구/music-discern/Datasets/')

#Urbansound is nested in Datasets/Urbansound8K/audio/fold**
#Urbansound is already separated into batches, which need to be preserved.
urban_metadata_path = pathlib.Path(base_path/'Urbansound8K/metadata')
urban_audio_folders_path = pathlib.Path(base_path/'Urbansound8K/audio')
urban_audio_path_list = [x for x in urban_audio_folders_path.iterdir()][1:] #remove .DS_Store

#Jamendo is nested in Datasets/MTGJamendo/audio
#Jamendo is not separated in batches, so we need to separate it into 10 batches, matching urbansound.
jamendo_metadata_path = pathlib.Path(base_path/'MTGJamendo/metadata')
jamendo_audio_path = pathlib.Path(base_path/'MTGJamendo/audio')

#for later -> add a path for any .wav file that I want to observe.
test_subjects_path = pathlib.Path("./")


# Loading and Checking audio/metadata files

### Metadata

In [29]:
print(sorted(urban_metadata_path.glob("*")))
print(sorted(jamendo_metadata_path.glob("*")))

[WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/metadata/.DS_Store'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/metadata/UrbanSound8K.csv')]
[WindowsPath('D:/전재원졸업연구/music-discern/Datasets/MTGJamendo/metadata/raw_30s_cleantags.tsv')]


In [43]:
urban_tags = pd.read_csv(sorted(urban_metadata_path.glob('*'))[1])
urban_tags.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [50]:
#jamendo_tags = pd.read_csv(sorted(jamendo_metadata_path.glob('*'))[0], sep="\t")
#cannot be read in this way, need a special script given by MTG-jamendo on Github
import csv
from collections import defaultdict

CATEGORIES = ['genre', 'instrument', 'mood/theme']
TAG_HYPHEN = '---'
METADATA_DESCRIPTION = 'TSV file with such columns: TRACK_ID, ARTIST_ID, ALBUM_ID, PATH, DURATION, TAGS'

def get_id(value):
    return int(value.split('_')[1])

def get_length(values):
    return len(str(max(values)))

def read_file(tsv_file):
    tracks = {}
    tags = defaultdict(dict)

    # For statistics
    artist_ids = set()
    albums_ids = set()

    with open(tsv_file) as fp:
        reader = csv.reader(fp, delimiter='\t')
        next(reader, None)  # skip header
        for row in reader:
            track_id = get_id(row[0])
            tracks[track_id] = {
                'id': track_id,
                'artist_id': get_id(row[1]),
                'album_id': get_id(row[2]),
                'path': row[3],
                'duration': float(row[4]),
                'tags': row[5:],  # raw tags, not sure if will be used
            }
            tracks[track_id].update({category: set() for category in CATEGORIES})

            artist_ids.add(get_id(row[1]))
            albums_ids.add(get_id(row[2]))

            for tag_str in row[5:]:
                category, tag = tag_str.split(TAG_HYPHEN)

                if tag not in tags[category]:
                    tags[category][tag] = set()

                tags[category][tag].add(track_id)

                if category not in tracks[track_id]:
                    tracks[track_id][category] = set()

                tracks[track_id][category].update(set(tag.split(",")))

    print("Reading: {} tracks, {} albums, {} artists".format(len(tracks), len(albums_ids), len(artist_ids)))

    extra = {
        'track_id_length': get_length(tracks.keys()),
        'artist_id_length': get_length(artist_ids),
        'album_id_length': get_length(albums_ids)
    }
    return tracks, tags, extra

(tracks,_,_) = read_file(sorted(jamendo_metadata_path.glob('*'))[0])

Reading: 55701 tracks, 11280 albums, 3573 artists


In [59]:
jamendo_tags = pd.DataFrame.from_dict(tracks,orient='index')
jamendo_tags.head()

,id,artist_id,album_id,path,duration,tags,genre,instrument,mood/theme
214,214,14,31,14/214.mp3,124.6,[genre---punkrock],{punkrock},{},{}
215,215,14,31,15/215.mp3,151.4,[genre---metal],{metal},{},{}
216,216,14,31,16/216.mp3,234.9,[genre---metal],{metal},{},{}
217,217,14,31,17/217.mp3,127.9,[genre---punkrock],{punkrock},{},{}
218,218,14,31,18/218.mp3,180.7,[genre---punkrock],{punkrock},{},{}


### Audio

# Setting Preprocessing variables